In [1]:
import tensorflow as tf
import datetime

import numpy as np
import matplotlib.pyplot as plt

2023-05-10 22:34:00.592219: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Prepare Datasets

 * Load train dataset and split into train and validation sets
 * Load test dataset
 * (Augment train dataset using randomized image manipulation) **INACTIVE**

In [2]:
train, validation = tf.keras.utils.image_dataset_from_directory(
    '../Dataset/train',
    labels='inferred',
    label_mode='categorical',
    image_size=(224,224),
    seed=815,
    validation_split=0.1,
    subset='both'
)

test = tf.keras.utils.image_dataset_from_directory(
    '../Dataset/test',
    labels='inferred',
    label_mode='categorical',
    image_size=(224,224)
)

def prepare(dataset):
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.RandomRotation(0.2),
        tf.keras.layers.RandomTranslation(0.2, 0.2),
        tf.keras.layers.RandomZoom(0.2),
    ])

    augmented = dataset
    for _ in range(4):
        augmented = augmented.concatenate(
            dataset.map(
                lambda x, y: (data_augmentation(x, training=True), y),
                num_parallel_calls=tf.data.AUTOTUNE
            )
        )
        
    return augmented.prefetch(buffer_size=tf.data.AUTOTUNE)


#train = prepare(train)

Found 22555 files belonging to 25 classes.
Using 20300 files for training.
Using 2255 files for validation.
Found 2500 files belonging to 25 classes.


2023-05-10 22:34:02.363898: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-10 22:34:02.367453: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-10 22:34:02.367606: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

### Create model
 * Instantiate base model (MobileNetV3Small) using ImageNet weights
 * Freeze layers of base model
 * Create new sequential model
   * First layer is base model
   * Following layers are dense with
     1. 128 Units, ReLU-Activation and L1L2 regularizer
     2. 128 Units, ReLU-Activation and L1L2 regularizer
     3. 25 Units, Softmax-Activation and L1L2 regularizer (output)
 * Compile model with adam optimizer
   * Use exponential learning rate decay

In [3]:
def generate_model():
    base = tf.keras.applications.MobileNetV3Small(
        include_top=False,
        weights='imagenet',
        input_shape=(224,224,3),
        pooling=None
    )
    
    for layer in base.layers:
        layer.trainable = False
    
    model = tf.keras.models.Sequential()
    model.add(base)

    regularizer = tf.keras.regularizers.L1L2(
        l1=0.00001,
        l2=0.005,
    )
    
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizer))
    model.add(tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizer))
    model.add(tf.keras.layers.Dense(25, activation='softmax', kernel_regularizer=regularizer))

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.001, 
        decay_steps=250, 
        decay_rate=0.9
    )
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
    
    return model

model = generate_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3small (Functiona  (None, 7, 7, 576)        939120    
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 28224)             0         
                                                                 
 dense (Dense)               (None, 128)               3612800   
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 25)                3225      
                                                                 
Total params: 4,571,657
Trainable params: 3,632,537
Non-trainable params: 939,120
________________________________________

### Transfer learning

Use train and validation dataset to fit dense layers to new dataset. Output data to TensorBoard and Checkpoint.

In [4]:
name = 'training-final'
now = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=f'logs/fit/{name}-{now}',
    write_steps_per_second=True
)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=f'checkpoints/{name}-{now}',
    monitor='val_accuracy',
)
    
model.fit(
    train,
    validation_data=validation,
    epochs=25, 
    callbacks=[
        tensorboard_callback,
        checkpoint_callback,
    ]
)

Epoch 1/25


2023-05-10 22:34:04.078833: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [20300]
	 [[{{node Placeholder/_4}}]]
2023-05-10 22:34:04.079074: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [20300]
	 [[{{node Placeholder/_0}}]]
/usr/local/lib/python3.8/dist-packages/keras/backend.py:5561: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2023-05-10 22:34:06.3826

635/635 [==============================] - ETA: 0s - loss: 2.7479 - accuracy: 0.6770

2023-05-10 22:34:19.144398: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2255]
	 [[{{node Placeholder/_0}}]]
2023-05-10 22:34:19.144738: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2255]
	 [[{{node Placeholder/_4}}]]


INFO:tensorflow:Assets written to: checkpoints/training-final-20230510-223404/assets


...

INFO:tensorflow:Assets written to: checkpoints/training-final-20230510-223404/assets


635/635 [==============================] - 21s 33ms/step - loss: 0.2564 - accuracy: 0.9991 - val_loss: 0.7006 - val_accuracy: 0.8568
Epoch 25/25
632/635 [============================>.] - ETA: 0s - loss: 0.2559 - accuracy: 0.9991

INFO:tensorflow:Assets written to: checkpoints/training-final-20230510-223404/assets


INFO:tensorflow:Assets written to: checkpoints/training-final-20230510-223404/assets


635/635 [==============================] - 21s 33ms/step - loss: 0.2560 - accuracy: 0.9991 - val_loss: 0.6998 - val_accuracy: 0.8563


### Fine tuning

 * Unfreeze layers of base model
 * Recompile model using adam optimizer and small static learning rate
 * Fit model to dataset

In [5]:
for layer in model.layers[0].layers:
    layer.Trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.000025),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=f'checkpoints/{name}-tune-{now}',
    monitor='val_accuracy',
)

model.fit(
    train,
    validation_data=validation,
    epochs=10, 
    callbacks=[
        tensorboard_callback,
        checkpoint_callback,
    ]
)

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/backend.py:5561: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


635/635 [==============================] - ETA: 0s - loss: 0.2679 - accuracy: 0.9978

INFO:tensorflow:Assets written to: checkpoints/training-final-tune-20230510-223404/assets


INFO:tensorflow:Assets written to: checkpoints/training-final-tune-20230510-223404/assets


...

INFO:tensorflow:Assets written to: checkpoints/training-final-tune-20230510-223404/assets


635/635 [==============================] - 21s 34ms/step - loss: 0.2558 - accuracy: 0.9976 - val_loss: 0.7073 - val_accuracy: 0.8581
Epoch 10/10
632/635 [============================>.] - ETA: 0s - loss: 0.2549 - accuracy: 0.9974

INFO:tensorflow:Assets written to: checkpoints/training-final-tune-20230510-223404/assets


INFO:tensorflow:Assets written to: checkpoints/training-final-tune-20230510-223404/assets


635/635 [==============================] - 22s 34ms/step - loss: 0.2549 - accuracy: 0.9974 - val_loss: 0.7051 - val_accuracy: 0.8603


### Evaluate model

Use test dataset to evaluate final performance of model

In [6]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=f'logs/evaluate/{name}-tune-{now}',
    write_steps_per_second=True
)

model.evaluate(
    test,
    callbacks=[tensorboard_callback]
)

 7/79 [=>............................] - ETA: 1s - loss: 0.8680 - accuracy: 0.8393

2023-05-10 22:46:40.138605: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2500]
	 [[{{node Placeholder/_0}}]]
2023-05-10 22:46:40.138932: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2500]
	 [[{{node Placeholder/_0}}]]


79/79 [==============================] - 2s 22ms/step - loss: 0.9514 - accuracy: 0.8144


[0.9513553977012634, 0.8144000172615051]

In [18]:
class_names = test.class_names

correct_predictions = dict.fromkeys(class_names, 0)
for images, labels in test:
    predictions = model.predict(images)
    for i in range(len(predictions)):
        label_class = np.argmax(labels[i])
        prediction_class = np.argmax(predictions[i])
    
        if label_class == prediction_class:
            correct_predictions[class_names[label_class]] += 1
        
for k in correct_predictions:
    print(f'{k};{correct_predictions[k] / 100}')
    
    

1/1 [==============================] - 0s 14ms/step
apple;0.78
apricot;0.9
avocado;0.84
banana;0.9
bell pepper;0.88
black berry;0.93
blueberry;0.94
cantaloupe;0.89
cherry;0.77
clementine;0.23
eggplant;0.96
grape;0.78
grapefruit;0.49
kiwi;0.97
lemon;0.73
lime;0.9
mandarine;0.83
nectarine;0.51
orange;0.62
pear;0.79
pineapple;0.98
pomegranate;0.86
raspberry;0.96
strawberry;0.96
watermelon;0.96
